In [ ]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 4.7 MB 13.1 MB/s 
     |████████████████████████████████| 1.1 MB 52.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
     |████████████████████████████████| 163 kB 44.9 MB/s 


In [ ]:
!pip install -U spacy-nightly --pre
!pip install transformers
!python -m spacy download en_core_web_trf
!pip install -U spacy transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.7 MB 8.0 MB/s 
     |████████████████████████████████| 660 kB 47.4 MB/s 
     |████████████████████████████████| 9.1 MB 46.9 MB/s 
  Attempting uninstall: typer
    Found existing installation: typer 0.4.2
    Uninstalling typer-0.4.2:
      Successfully uninstalled typer-0.4.2
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.9.2
    Uninstalling pydantic-1.9.2:
      Successfully uninstalled pydantic-1.9.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.4
    Uninstalling thinc-8.1.4:
      Successfully uninstalled thinc-8.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.4.1 requires thinc<8.2.0,>=8.1.0, but you have thinc 8.0.17 which is incompatible.
Looki

In [ ]:
!pip install -U pip setuptools wheel
!python -m spacy project clone tutorials/rel_component

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.2 MB/s 
     |████████████████████████████████| 1.2 MB 49.3 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
spacy-transformers 1.0.4 requires transformers<4.10.0,>=3.4.0, but you have transformers 4.23.1 which is incompatible.
spacy-nightly 3.0.0rc5 requires thinc<8.1.0,>=8.0.0, but you have thinc 8.1.5 which is incompatible.
en-core-web-trf 3.

✔ Cloned 'tutorials/rel_component' from 'explosion/projects' (branch
'v3')
/content/rel_component
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/rel_component


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import transformers
from datasets import load_dataset
import pandas as pd
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification
import numpy as np
import spacy
from spacy.tokens import Span, DocBin, Doc

In [ ]:
dataset_entities = load_dataset("ade_corpus_v2",'Ade_corpus_v2_drug_ade_relation')
dataset_classification = load_dataset("ade_corpus_v2",'Ade_corpus_v2_classification')

Generating train split:   0%|          | 0/6821 [00:00<?, ? examples/s]

Dataset ade_corpus_v2 downloaded and prepared to /root/.cache/huggingface/datasets/ade_corpus_v2/Ade_corpus_v2_drug_ade_relation/1.0.0/940d61334dbfac6b01ac5d00286a2122608b8dc79706ee7e9206a1edb172c559. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/23516 [00:00<?, ? examples/s]

Dataset ade_corpus_v2 downloaded and prepared to /root/.cache/huggingface/datasets/ade_corpus_v2/Ade_corpus_v2_classification/1.0.0/940d61334dbfac6b01ac5d00286a2122608b8dc79706ee7e9206a1edb172c559. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(dataset_entities)
print(dataset_classification)

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'effect', 'indexes'],
        num_rows: 6821
    })
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 23516
    })
})


In [ ]:
dataset_dict={}

for index,row in enumerate(dataset_entities["train"]):
  try:
    if row["text"] in dataset_dict:
      e = {
                      "text": row["drug"],
                      "start": row["indexes"]["drug"]["start_char"][0],
                      "end": row["indexes"]["drug"]["end_char"][0],
                      "entityLabel": "DRUG"
              }

      if e not in dataset_dict[row["text"]]["tokens"]:
        dataset_dict[row["text"]]["tokens"].append({
                        "text": row["drug"],
                        "start": row["indexes"]["drug"]["start_char"][0],
                        "end": row["indexes"]["drug"]["end_char"][0],
                        "entityLabel": "DRUG"
                }) 
        
      e = {
                      "text": row["effect"],
                      "start": row["indexes"]["effect"]["start_char"][0],
                      "end": row["indexes"]["effect"]["end_char"][0],
                      "entityLabel": "EFFECT"
              }
      if e not in dataset_dict[row["text"]]["tokens"]:
        dataset_dict[row["text"]]["tokens"].append({
                        "text": row["effect"],
                        "start": row["indexes"]["effect"]["start_char"][0],
                        "end": row["indexes"]["effect"]["end_char"][0],
                        "entityLabel": "EFFECT"
                }) 
      dataset_dict[row["text"]]["relations"].append(  { "child": 4, "head": 0, "relationLabel": "ADVERSE REACTION", "start_drug":row["indexes"]["drug"]["start_char"][0], "start_effect":row["indexes"]["effect"]["start_char"][0], "drug":row["drug"] , "effect":row["effect"] }) 

    else :
      dataset_dict[row["text"]] = {
              "tokens": [{
                      "text": row["drug"],
                      "start": row["indexes"]["drug"]["start_char"][0],
                      "end": row["indexes"]["drug"]["end_char"][0],
                      "entityLabel": "DRUG"
              },{
                      "text": row["effect"],
                      "start": row["indexes"]["effect"]["start_char"][0],
                      "end": row["indexes"]["effect"]["end_char"][0],
                      "entityLabel": "EFFECT"
              }
              ],
              "document":row["text"],
              "relations": [
                    { "child": 4, "head": 0, "relationLabel": "ADVERSE REACTION", "start_drug":row["indexes"]["drug"]["start_char"][0], "start_effect":row["indexes"]["effect"]["start_char"][0], "drug":row["drug"] , "effect":row["effect"] },
              ] }
  except:
    continue
    print(row)      

In [ ]:
#nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm")
tokens=nlp("MS/Ph.D. in CS/EE or related areas is required. Strong ability and effectiveness working")    
words = [t.text for t in tokens]



In [ ]:
from spacy.lang.en import English
from spacy.tokenizer import Tokenizer

In [ ]:
#nlp = spacy.load('en_core_web_sm', disable = ['ner', 'parser', 'tagger'])
#nlp.add_pipe('sentencizer')
from tqdm import tqdm

for key,content in tqdm(dataset_dict.items()):
  tokens= nlp(content['document'])
  #print(tokens)
  words = [t.text for t in tokens]
  #print(words)

  child = ""
  head = ""
  

  pair_nb = 0
  for entity in content['tokens']:
    entity_tokens = nlp(entity['text'])

    #print("Working on entity:",entity_tokens)
    start = ""
    end = ""
    for i,sent in enumerate(tokens.sents):
      for i,token in enumerate(sent):
          #print(token,token.i,token.idx)
          if(token.idx == entity['start'] and token.text in entity['text']):
            #print(token.text, token.i,token.idx)
            start = token.i
            end = token.i + len(entity_tokens) - 1

            for relation in content['relations']:
              #print(relation,entity)
              if entity['entityLabel'] == "DRUG" and entity['start']== relation['start_drug']:
                relation['child']  = start
              if entity['entityLabel'] == "EFFECT" and entity['start']== relation['start_effect']:
                relation['head'] = start

    entity['token_start'] = start
    entity['token_end'] = end
    

    

100%|██████████| 4258/4258 [01:23<00:00, 50.93it/s]


In [ ]:
for key,content in dataset_dict.items():
  print(content)
  break

{'tokens': [{'text': 'azithromycin', 'start': 12, 'end': 24, 'entityLabel': 'DRUG', 'token_start': 1, 'token_end': 1}, {'text': 'ototoxicity', 'start': 33, 'end': 44, 'entityLabel': 'EFFECT', 'token_start': 4, 'token_end': 4}], 'document': 'Intravenous azithromycin-induced ototoxicity.', 'relations': [{'child': 1, 'head': 4, 'relationLabel': 'ADVERSE REACTION', 'start_drug': 12, 'start_effect': 33, 'drug': 'azithromycin', 'effect': 'ototoxicity'}]}


In [ ]:
data_negative = [e for e in dataset_classification["train"] if e['label']==0]

In [ ]:
len(data_negative)

16695

In [ ]:
NER = spacy.load("/content/drive/MyDrive/Colab Notebooks/NER")

In [ ]:
data_negative[50]

{'text': 'Significance of subclinical entrapment of nerves in lead neuropathy.',
 'label': 0,
 'has_reaction': True,
 'tokens': [{'text': 'lead',
   'token_start': 7,
   'token_end': 8,
   'start': 52,
   'end': 56,
   'entityLabel': 5792857178182177913},
  {'text': 'neuropathy',
   'token_start': 8,
   'token_end': 9,
   'start': 57,
   'end': 67,
   'entityLabel': 10800358000381068841}]}

In [ ]:
for sent in tqdm(data_negative):
  doc = NER(sent["text"])
  entities = []
  adverse_found = False
  drug_found = False
  for e in doc.ents:
    if e.label_ == "EFFECT":
      adverse_found = True
    entities.append({'text':e.text,'token_start':e.start,'token_end':e.end,'start':e.start_char,'end':e.end_char,'entityLabel': e.label_})
  if adverse_found :
    sent['has_reaction'] = True
    sent['tokens'] = entities




100%|██████████| 16695/16695 [34:29<00:00,  8.07it/s]


In [ ]:
data_negative_with_reaction = [e for e in data_negative if "has_reaction" in e]
len(data_negative_with_reaction)

10461

In [ ]:
for e in data_negative_with_reaction:
  relations=[]
  drug_found = False
  for entity in e['tokens']:
    if entity['entityLabel'] == "EFFECT":
      drug_found = True
    if entity['entityLabel'] == "EFFECT":
      for entity_b in e['tokens']:
        if entity_b['entityLabel'] == "DRUG":
          relations.append({'child': entity_b['token_start'], 'head': entity['token_start'], 'relationLabel': 'NOT ADVERSE REACTION'})
  e['relations'] = relations

In [ ]:
#Removing entries with no drugs (only reactions)
for e in data_negative_with_reaction[:]:
  drug_found = False
  for entity in e['tokens']:
    if entity['entityLabel'] == "DRUG":
      drug_found = True
  if not drug_found:
    data_negative_with_reaction.remove(e)


In [ ]:
data_negative_with_reaction[2]

{'text': 'Digoxin serum levels and toxicity risks increase with the addition of amiodarone.',
 'label': 0,
 'has_reaction': True,
 'tokens': [{'text': 'Digoxin',
   'token_start': 0,
   'token_end': 1,
   'start': 0,
   'end': 7,
   'entityLabel': 'EFFECT'},
  {'text': 'serum levels',
   'token_start': 1,
   'token_end': 3,
   'start': 8,
   'end': 20,
   'entityLabel': 'EFFECT'},
  {'text': 'toxicity risks',
   'token_start': 4,
   'token_end': 6,
   'start': 25,
   'end': 39,
   'entityLabel': 'EFFECT'},
  {'text': 'amiodarone',
   'token_start': 11,
   'token_end': 12,
   'start': 70,
   'end': 80,
   'entityLabel': 'DRUG'}],
 'relations': [{'child': 11,
   'head': 0,
   'relationLabel': 'NOT ADVERSE REACTION'},
  {'child': 11, 'head': 1, 'relationLabel': 'NOT ADVERSE REACTION'},
  {'child': 11, 'head': 4, 'relationLabel': 'NOT ADVERSE REACTION'}]}

In [ ]:
for e in data_negative_with_reaction:
  e['document'] = e['text']
  dataset_dict[e['text']] = e

In [ ]:
test_dict={}
for e in data_negative_with_reaction:
  test_dict[e['text']] = e

In [ ]:
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/Relations/training_data.pickle', 'wb') as handle:
    pickle.dump(dataset_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('filename.pickle', 'rb') as handle:
    #b = pickle.load(handle)


In [ ]:
Doc.set_extension("rel", default={},force=True)
vocab = Vocab()

SYMM_LABELS = ["Binds"]
MAP_LABELS = {
      "ADVERSE REACTION": "ADVERSE REACTION",
      "NOT ADVERSE REACTION": "NOT ADVERSE REACTION",
  }

docs = {"train": [], "dev": [], "test": [], "total": []}
ids = {"train": set(), "dev": set(), "test": set(), "total":set()}
count_all = {"train": 0, "dev": 0, "test": 0,"total": 0}
count_pos = {"train": 0, "dev": 0, "test": 0,"total": 0}

i=0
error = 0
for key,content in tqdm(dataset_dict.items()):
  try:
    example = content
    span_starts = set()
    neg = 0
    pos = 0
    # Parse the tokens
    tokens=nlp(example["document"])
    spaces=[]
    spaces = [True if tok.whitespace_ else False for tok in tokens]
    words = [t.text for t in tokens]
    doc = Doc(nlp.vocab, words=words, spaces=spaces)

    spans = example["tokens"]
    entities = []
    span_end_to_start = {}
    for span in spans:
        entity = doc.char_span(
              span["start"], span["end"], label=span["entityLabel"]
          )


        span_end_to_start[span["token_start"]] = span["token_start"]
        #print(span_end_to_start)
        entities.append(entity)
        span_starts.add(span["token_start"])

    doc.ents = entities

    # Parse the relations
    rels = {}
    for x1 in span_starts:
        for x2 in span_starts:
            rels[(x1, x2)] = {}
            #print(rels)
    relations = example["relations"]
    #print(len(relations))
    for relation in relations:
        # the 'head' and 'child' annotations refer to the end token in the span
        # but we want the first token
        start = span_end_to_start[relation["head"]]
        end = span_end_to_start[relation["child"]]
        label = relation["relationLabel"]
        #print(rels[(start, end)])
        #print(label)
        #label = MAP_LABELS[label]
        if label not in rels[(start, end)]:
            rels[(start, end)][label] = 1.0
            pos += 1
            #print(pos)
            #print(rels[(start, end)])


    for x1 in span_starts:
        for x2 in span_starts:
            for label in MAP_LABELS.values():
                if label not in rels[(x1, x2)]:
                    neg += 1
                    rels[(x1, x2)][label] = 0.0

                    #print(rels[(x1, x2)])
    doc._.rel = rels

    if pos > 0:
      docs["total"].append(doc)
      count_pos["total"] += pos
      count_all["total"] += pos + neg
  except Exception as e:
    error += 1
    pass



docbin = DocBin(docs=docs["total"], store_user_data=True)
#docbin.to_disk("relations_training.spacy")
print(f"{error} removed from dataset")
print(f"{len(docs['total'])} training sentences")

100%|██████████| 8211/8211 [01:09<00:00, 118.36it/s]


153 removed from dataset
8058 training sentences


In [ ]:
import random
random.shuffle(docs["total"])
len(docs["total"])

8058

In [ ]:
train = docs["total"][0:7000]
test = docs["total"][7001:]

In [ ]:
docbin = DocBin(docs=train, store_user_data=True)
docbin.to_disk("relations_training.spacy")

In [ ]:
docbin = DocBin(docs=test, store_user_data=True)
docbin.to_disk("relations_dev.spacy")

In [ ]:
!cd rel_component

In [ ]:
import spacy_transformers

In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd rel_component

/content/rel_component


In [ ]:
from scripts.rel_pipe import make_relation_extractor, score_relations
from scripts.rel_model import create_relation_model, create_classification_layer, create_instances


In [ ]:
!spacy project run train_gpu


================================= train_gpu =================================
Running command: /usr/bin/python3 -m spacy train configs/rel_trf.cfg --output training --paths.train data/relations_training.spacy --paths.dev data/relations_dev.spacy -c ./scripts/custom_functions.py --gpu-id 0
ℹ Saving to output directory: training
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-10-24 18:52:47,877] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-10-24 18:52:47,886] [INFO] Pipeline: ['transformer', 'relation_extractor']
INFO:spacy:Pipeline: ['transformer', 'relation_extractor']
[2022-10-24 18:52:47,890] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-10-24 18:52:47,891] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Downloading: 100% 1.11k/1.11k [00:00<00:00, 1.49MB/s]
Downloading: 100% 213k/213k [00:00<00:00, 865kB/s]
Downloading: 100% 436M/436M [00:20<

In [ ]:
!spacy project run evaluate


================================== evaluate ==================================
Running command: /usr/bin/python3 ./scripts/evaluate.py training/model-best data/relations_dev.spacy False
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
Moving 0 files to the new cache system
0it [00:00, ?it/s]

Random baseline:
threshold 0.00 	 {'rel_micro_p': '17.10', 'rel_micro_r': '100.00', 'rel_micro_f': '29.20'}
threshold 0.05 	 {'rel_micro_p': '17.08', 'rel_micro_r': '95.07', 'rel_micro_f': '28.96'}
threshold 0.10 	 {'rel_micro_p': '17.04', 'rel_micro_r': '89.83', 'rel_micro_f': '28.65'}
threshold 0.20 	 {'rel_micro_p': '17.21', 'rel_micro_r': '80.04', 'rel_micro_f': '28.33'}
threshold 0.30 	 {'rel_micro_p': '16.92', 'rel_micro_r': '68.81', 'rel_micro_f': '27.16'}
threshold 0.40 	 {'rel_micro_p': '17.11', 'rel_micro_r

In [ ]:
nlp = spacy.load("/content/drive/MyDrive/Colab Notebooks/NER")

In [ ]:
text = ['''Tramadol, sold under the brand name Ultram among others, is an opioid pain medication used to treat moderate to moderately severe pain. When taken by mouth in an immediate-release formulation, the onset of pain relief usually begins within an hour. It is also available by injection. It is available in combination with paracetamol (acetaminophen). As is typical of opioids, common side effects include constipation, itchiness, and nausea. Serious side effects may include hallucinations, seizures, increased risk of serotonin syndrome, decreased alertness, and drug addiction. A change in dosage may be recommended in those with kidney or liver problems. It is not recommended in those who are at risk of suicide or in those who are pregnant. While not recommended in women who are breastfeeding, those who take a single dose should not generally stop breastfeeding. Tramadol is converted in the liver to O-desmethyltramadol (desmetramadol), an opioid with a stronger affinity to the μ-opioid receptor. Tramadol is also a serotonin–norepinephrine reuptake inhibitor (SNRI). Tramadol was patented in 1963 and launched under the name "Tramal" in 1977 by the West German pharmaceutical company Grünenthal GmbH.[15][16] In the mid-1990s, it was approved in the United Kingdom and the United States.[15] It is available as a generic medication and marketed under many brand names worldwide.In 2020, it was the 35th most commonly prescribed medication in the United States, with more than 17 million prescriptions.''']

In [ ]:
for doc in nlp.pipe(text, disable=["tagger"]):
   print(f"spans: {[(e.start, e.text, e.label_) for e in doc.ents]}")

spans: [(0, 'Tramadol', 'DRUG'), (60, 'paracetamol', 'DRUG'), (62, 'acetaminophen', 'DRUG'), (75, 'constipation', 'EFFECT'), (77, 'itchiness', 'EFFECT'), (80, 'nausea', 'EFFECT'), (87, 'hallucinations', 'EFFECT'), (89, 'seizures', 'EFFECT'), (97, 'decreased alertness', 'EFFECT'), (101, 'drug addiction', 'EFFECT'), (159, 'Tramadol', 'DRUG'), (166, 'O-desmethyltramadol', 'DRUG'), (170, 'desmetramadol', 'DRUG'), (186, 'Tramadol', 'DRUG'), (196, 'SNRI', 'EFFECT'), (199, 'Tramadol', 'DRUG')]


In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True,options={
        "colors": {
                   "DRUG": "#f08080",
                   "EFFECT": "#9bddff",
                   
               },
    })

In [ ]:
nlp2 = spacy.load("training/model-best")
nlp2.add_pipe('sentencizer')

In [ ]:
for name, proc in nlp2.pipeline:
  doc = proc(doc)

In [ ]:
for value, rel_dict in doc._.rel.items():
  for sent in doc.sents:
    for e in sent.ents:
      for b in sent.ents:
        if e.start == value[0] and b.start == value[1]:
          if rel_dict['ADVERSE REACTION'] >=0.5 or rel_dict['NOT ADVERSE REACTION'] >=0.5:
            print(f"{sent.text, e.text, b.text} --> predicted relation: {rel_dict}")

('Tramadol is also a serotonin–norepinephrine reuptake inhibitor (SNRI).', 'SNRI', 'Tramadol') --> predicted relation: {'ADVERSE REACTION': 0.0030176162, 'NOT ADVERSE REACTION': 0.9499226}


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!cp /content/NER /drive/MyDrive

cp: -r not specified; omitting directory '/content/NER'


In [ ]:
!zip -r /content/training/relationship_extractor.zip /content/training/model-last

  adding: content/training/model-last/ (stored 0%)
  adding: content/training/model-last/config.cfg (deflated 64%)
  adding: content/training/model-last/meta.json (deflated 54%)
  adding: content/training/model-last/tokenizer (deflated 81%)
  adding: content/training/model-last/vocab/ (stored 0%)
  adding: content/training/model-last/vocab/vectors.cfg (stored 0%)
  adding: content/training/model-last/vocab/vectors (deflated 45%)
  adding: content/training/model-last/vocab/lookups.bin (stored 0%)
  adding: content/training/model-last/vocab/key2row (stored 0%)
  adding: content/training/model-last/vocab/strings.json (deflated 72%)
  adding: content/training/model-last/transformer/ (stored 0%)
  adding: content/training/model-last/transformer/cfg (stored 0%)
  adding: content/training/model-last/transformer/model/ (stored 0%)
  adding: content/training/model-last/transformer/model/tokenizer.json (deflated 71%)
  adding: content/training/model-last/transformer/model/vocab.txt (deflated 52%

In [ ]:
!zip -r /content/NER.zip /content/NER

  adding: content/NER/ (stored 0%)
  adding: content/NER/ner/ (stored 0%)
  adding: content/NER/ner/cfg (deflated 33%)
  adding: content/NER/ner/model (deflated 8%)
  adding: content/NER/ner/moves (deflated 52%)
  adding: content/NER/config.cfg (deflated 60%)
  adding: content/NER/meta.json (deflated 57%)
  adding: content/NER/tokenizer (deflated 81%)
  adding: content/NER/NER/ (stored 0%)
  adding: content/NER/NER/NER/ (stored 0%)
  adding: content/NER/NER/NER/transformer/ (stored 0%)
  adding: content/NER/NER/NER/transformer/cfg (stored 0%)
  adding: content/NER/NER/NER/transformer/model/ (stored 0%)
  adding: content/NER/NER/NER/transformer/model/tokenizer.json (deflated 71%)
  adding: content/NER/NER/NER/transformer/model/vocab.txt (deflated 52%)
  adding: content/NER/NER/NER/transformer/model/tokenizer_config.json (deflated 46%)
  adding: content/NER/NER/NER/transformer/model/special_tokens_map.json (deflated 42%)
  adding: content/NER/NER/NER/transformer/model/config.json (deflat

In [ ]:
import random
import typer
from pathlib import Path
import spacy
from spacy.tokens import DocBin, Doc
from spacy.training.example import Example
from scripts.rel_pipe import make_relation_extractor, score_relations
from scripts.rel_model import create_relation_model, create_classification_layer, create_instances, create_tensors

In [ ]:
nlp2 = spacy.load("training/model-best")

In [ ]:
for name, proc in nlp2.pipeline:
  print(name,proc)
  doc = proc(doc)

transformer <spacy_transformers.pipeline_component.Transformer object at 0x7f4d605ed280>
relation_extractor <scripts.rel_pipe.RelationExtractor object at 0x7f4d606028a0>


In [ ]:
doc

Successful challenge with clozapine in a history of eosinophilia.